In [1]:
!pip install -U datasets
!pip install wandb
!pip install torch
!pip install Cython

!pip install torch torchvision torchaudio
!pip install -U transformers
!pip install peft
!pip install -U bitsandbytes
!pip install tensorboard
!pip install accelerate -U
!pip install pandas
!pip install tqdm
from tqdm import tqdm

In [2]:
from datetime import datetime
import os
import sys
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

from peft import (
    PeftModel,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

base_model = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

model = PeftModel.from_pretrained(model, "dohonba/mistral_7b_fingpt")
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear8bitLt(
                (base_layer

In [ ]:
# eval_prompt = """
# Is this sentence self-promotional? Answer with {no/yes}? "Building brick by brick, our analysts motto! Pay a visit to our Community".
# """

# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# with torch.no_grad():
#     output = model.generate(**model_input, max_new_tokens=150)[0]
#     decoded_output = tokenizer.decode(output, skip_special_tokens=True)

# print(decoded_output)

In [10]:
# Function to classify emotion of a sentence
def classify_sentiment(sentence):
    eval_prompt = f"""Context: {sentence}

Question: 'What is the sentiment of this sentence? Please choose an answer from {{strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}}.'
"""
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(**model_input, max_new_tokens=150)[0]
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)

    answer = decoded_output.split("Answer: ", 1)[1]
    return answer  # You might need to further process this to extract the emotion

def classify_emotion(sentence):
    eval_prompt = f"""Context: {sentence}

Question: 'What is the emotion shown in this text? Please choose an answer from {{anger/fear/joy/love/sadness/surprise/neutral}}'.
"""
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(**model_input, max_new_tokens=150)[0]
        decoded_output = tokenizer.decode(output, skip_special_tokens=True)

    answer = decoded_output.split("Answer: ", 1)[1]
    return answer  # You might need to further process this to extract the emotion

In [13]:
classify_emotion("I love it. Thanks.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'love'

In [ ]:
article = NewsPlease.from_url('https://finance.yahoo.com/news/alaska-airlines-begun-flying-boeing-150009733.html')

# Split the article text into sentences
sentences = sent_tokenize(article.maintext)
print("Sentences in the article: ", len(sentences))

# Classify emotion for each sentence with a progress bar
emotion_results = []
for i, sentence in enumerate(tqdm(sentences, desc="Processing Sentences")):
    emotion = classify_sentiment(sentence)
    emotion_results.append(emotion)

# Do something with the results
print(emotion_results)

# print(article.maintext)

In [14]:
import json
from datetime import datetime, timedelta
import pandas as pd
import pytz

# Calculate the cutoff time for the last 30 minutes
cutoff_time = datetime.utcnow().replace(tzinfo=pytz.utc) - timedelta(minutes=30)

def load_recent_tweets(file_path, cutoff_time):
    recent_tweets = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            tweet = json.loads(line)
            tweet_time = datetime.fromisoformat(tweet['date'])  # Assuming UTC and removing 'Z'
            if tweet_time > cutoff_time:
                recent_tweets.append(tweet)  # Assuming text is under 'rawContent'
    return recent_tweets

def save_processed_tweets(file_path, tweets):
    with open(file_path, 'w', encoding='utf-8') as file:
        for tweet in tweets:
            # Create a new dictionary with only the required fields
            filtered_tweet = {
                'id': tweet['id'],  # Assuming each tweet has a unique 'id'
                'date': tweet['date'],
                'cleanContent': tweet.get('cleanContent', ''),
                'emotion': tweet.get('emotion', ''),
                'sentiment': tweet.get('sentiment', '')
            }
            json.dump(filtered_tweet, file)
            file.write('\n')


tweets = load_recent_tweets('english_tweets.jsonl', cutoff_time)
print(len(tweets))

from tqdm import tqdm

limit = 10
sentiment_results = []
emotion_results = []
for i, tweet in enumerate(tqdm(tweets[:limit], desc="Analyzing Tweets")):
    clean_content = tweet.get('cleanContent', '')
    
    sentiment = classify_sentiment(clean_content)
    emotion = classify_emotion(clean_content)
    
    sentiment_results.append(f"{i}: " + sentiment)
    emotion_results.append(f"{i}: " + emotion)
    
    tweet['sentiment'] = sentiment
    tweet['emotion'] = emotion

#Save inference
save_processed_tweets('processed_english_tweets.jsonl', tweets)

# Print results
print(sentiment_results)
print(emotion_results)
for i, tweet in enumerate(tweets):
    print(f"{i}: " + tweet.get('cleanContent', ''))

61


Analyzing Tweets:   0%|          | 0/10 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  10%|█         | 1/10 [00:03<00:29,  3.32s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  20%|██        | 2/10 [00:06<00:27,  3.49s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  30%|███       | 3/10 [00:10<00:24,  3.56s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzing Tweets:  40%|████      | 4/10 [00:14<00:21,  3.59s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Analyzin

['0: strong positive', '1: moderately positive', '2: mildly positive', '3: moderately positive', '4: strong negative', '5: moderately positive', '6: neutral', '7: moderately positive', '8: moderately positive', '9: mildly positive']
['0: neutral', '1: neutral', '2: neutral', '3: neutral', '4: anger', '5: neutral', '6: neutral', '7: neutral', '8: neutral', '9: neutral']
0: If you invested in on that day, you’d currently be up over 8,800%
1: Top analyst price target for next week..
2: “.. functions better when he has a woman he loves telling him when he is being an idiot. Like all of us!” Printed &amp; framed!
3: Nvdidia bull flag! The bull flags I predicted / pointed out yesterday for SMCI and Tesla are playing out beautifully BTW
4: I still cannot believe Sony stole and patented MY INVENTION I created when I was a kid. So far two of my inventions have been stolen and other people are profiting off. Doing me dirty like Tesla and I was only in Elementary School
5: The time has come for t

In [ ]:
!pip3 install news-please
from newsplease import NewsPlease

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize